In [1]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [2]:
%load_ext autoreload

%autoreload 2

In [5]:
import torch
from glasses.nn.models.segmentation.unet import UNet, UNetEncoder
from glasses.nn.models.classification.senet import SENetBasicBlock
from glasses.nn.models.classification.resnet import ResNetEncoder, ResNet
from glasses.nn.blocks import Conv2dPad
from functools import partial
x = torch.rand((1, 1, 32 * 12, 32*12))
# custom encoder
resnet = ResNet.resnet18(in_channels=1)
# we need to change the first conv in order to accept a gray image
# resnet.encoder.blocks[0].block[0].block.block.conv1 = Conv2dPad(1, 64, kernel_size=1)
# unet = UNet(1, n_classes=2, widths=resnet.encoder.widths)
# use resnet18 as encoder
resnet = ResNet.resnet18(in_channels=1)
unet = UNet(1, n_classes=2, widths=resnet.encoder.widths)
unet.encoder = resnet.encoder
unet(x)

unet = UNet(encoder=partial(UNetEncoder, block=SENetBasicBlock))
unet(x)

encoder = ResNetEncoder(in_channels=1, widths=[64,128,256,512,1024], depths=[2,2,2,2,2])
unet = UNet(1, n_classes=2, widths=encoder.widths)
unet.encoder = encoder
unet(x)
# # custom block
# unet = UNet(down_block=SENetBasicBlock, up_block=SENetBasicBlock)
# unet(x)


tensor([[[[-0.5097, -0.1584, -0.1039,  ..., -0.4699, -0.4895, -0.3134],
          [-0.3268, -0.0977,  0.5717,  ...,  0.1253, -0.2982,  0.0250],
          [-0.3186,  0.0352,  0.6240,  ..., -0.3029, -0.3958, -0.6869],
          ...,
          [-0.4024,  0.4335,  0.2689,  ..., -0.9266, -0.0629, -0.7173],
          [ 0.0423, -0.1966,  0.1985,  ...,  0.1283,  0.2436,  0.0872],
          [-0.1668, -0.1652,  0.2772,  ..., -0.2744, -0.3322, -0.2125]],

         [[-0.2416, -0.1410, -0.3617,  ..., -0.4207, -0.4389,  0.0384],
          [-0.5749, -0.6739, -0.3115,  ..., -0.2171, -0.1255,  0.0023],
          [-0.1302, -0.2923, -0.1579,  ..., -0.1351, -0.0416, -0.3534],
          ...,
          [-0.0748, -0.3791,  0.0713,  ..., -0.2042, -0.3289,  0.3153],
          [-0.4919, -0.7480, -0.2047,  ..., -0.1089,  0.3773, -0.2863],
          [ 0.0320, -0.0969,  0.0226,  ...,  0.3943,  0.4132, -0.2187]]]],
       grad_fn=<MkldnnConvolutionBackward>)

In [ ]:
unet

In [ ]:
resnet.encoder

In [ ]:
from __future__ import annotations
import torch
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from collections import OrderedDict
from typing import List
from functools import partial
from glasses.nn.blocks import ConvBnAct, Conv2dPad
from glasses.nn.models.segmentation.unet import UNet


In [ ]:
from glasses.nn.models.classification import ResNet
from glasses.nn.models.segmentation.unet import  UNet

resnet34 = ResNet.resnet18()
# we need to change the first conv in order to accept a gray image
resnet34.encoder.blocks[0].block[0].block.block.conv1 = Conv2dPad(1, 64, kernel_size=1)

unet = UNet(1, n_classes=2, blocks_sizes=resnet34.encoder.blocks_sizes)

unet.encoder.blocks = resnet34.encoder.blocks

x = torch.rand((1, 1, 32 * 12, 32*12))
unet(x).shape

# unet.decoder

In [ ]:
from glasses.nn.models.classification.senet import SENetBasicBlock

block = SENetBasicBlock

unet = UNet(down_block=SENetBasicBlock, up_block=SENetBasicBlock)

x = torch.rand((1, 1, 32 * 12, 32*12))
unet(x).shape

In [ ]:
unet = UNet(n_classes=4)
unet(x).shape

In [ ]:
[r.shape for r in unet.residuals]

In [ ]:
from glasses.nn.models.classification.mobilenet import InvertedResidualBlock
from glasses.nn.models.classification.se import SEModule


In [ ]:
UpBlock(32, 64)